##**Pré-Processamento de Dados**
Tratando o Dataset BCICIV_2A

BCI Competition 2008 - Graz data set A

In [ ]:
# importações padrão
import numpy as np
import scipy.io

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ==============================================================
# Class and Functions
# ==============================================================
def defining_labels(events, true_labels):
  '''
    Essa função é dedicada para o conversão dos eventos desconhecidos
    em eventos conhecidos, através dos true labels.
    Args:
      events:(ndarray) array de eventos desconhecidos.
      true_labels:(ndarray) array de eventos conhecidos.
    Return:
      events:(ndarray) array com os eventos corretos definidos.
  '''
  counter = 0;
  for i in range(events.shape[0]):
    if events[i] == 783:
        if true_labels[counter] == 1:
          events[i] = 769
        elif true_labels[counter] == 2:
          events[i] = 770
        elif true_labels[counter] == 3:
          events[i] = 771
        elif true_labels[counter] == 4:
          events[i] = 772
        counter+=1

def extractor_epochs(eeg, events, position):
  '''
    Essa função é didicada para realizar a extração das épocas do sinal
    de eeg em que ocorrem as imagéticas motoras da mão esquerda e direita.
    Args:
      eeg:(ndarray) array com os sinais de eeg.
      events:(ndarray) array com os eventos (769 - mão esquerda, 770 - mão direita).
      position:(ndarray) array de posições dos eventos.
    Return:
      epochs_eeg:(ndarray) array com os sinais de eeg dividios em épocas.
  '''
  left_hand, right_hand, sample, trials = 0, 0, 500, 72
  epochs_eeg = np.zeros((144, 22, sample))
  for i in range(events.shape[0]):
      if events[i] == 769:
          epochs_eeg[left_hand] = eeg[:22,position[i]+125:position[i]+125+sample]
          left_hand += 1
      elif events[i] == 770:
          epochs_eeg[right_hand+trials] = eeg[:22,position[i]+125:position[i]+125+sample]
          right_hand += 1
  return epochs_eeg

In [ ]:
# ==============================================================
# SETUP
# ==============================================================
suj = 1

In [ ]:
# ==============================================================
# DATASET DE TREINAMENTO
# ==============================================================
# lendo o conjunto de dados de avaliação
data_train = scipy.io.loadmat('/content/drive/MyDrive/Colab Notebooks/learn_BCI/Dataset/Treino/A0' + str(suj) + 'T.mat')

# armazenandos os eventos desconhecidos
event_train = data_train['tipo_evento'].reshape(-1)

# armazenandos as posições
position_train = data_train['posicao'].reshape(-1)-1

# armazenando os sinais
eeg_train = data_train['s'].T

# extração de épocas
epochs_eeg_train = extractor_epochs(eeg_train, event_train, position_train)

# salvando os dados pré-processados
np.save('A0'+str(suj)+'T',epochs_eeg_train)

In [ ]:
epochs_eeg_train.shape

(144, 22, 500)

In [ ]:
# ==============================================================
# DATASET DE AVALIAÇÃO
# ==============================================================
# lendo o conjunto de dados de avaliação
data_test = scipy.io.loadmat('/content/drive/MyDrive/Colab Notebooks/learn_bcieeg/Dataset/Teste/A0' + str(suj) + 'E.mat')

# armazenandos os eventos desconhecidos
event_test = data_test['tipo_evento'].reshape(-1)

# armazenandos as posições
position_test = data_test['posicao'].reshape(-1)-1

# armazenando os sinais
eeg_test = data_test['s'].T

# definindo os eventos desconhecidos
true_labels = scipy.io.loadmat('/content/drive/MyDrive/Colab Notebooks/learn_bcieeg/Dataset/Labels/A0' + str(suj) + 'E.mat')['classlabel'].reshape(-1)
defining_labels(event_test, true_labels)

# extração de épocas
epochs_eeg_test = extractor_epochs(eeg_test, event_test, position_test)

# salvando os dados pré-processados
np.save('A0'+str(suj)+'E',epochs_eeg_test)